## 이 문서를 수정할 당신에게...
#### 현재 상황은 아래와 같습니다.
1. 아래 4가지의 도구를 사용하고 있습니다.
- 필터 query로 데이터베이스 정보 추출(추천을 위함): chroma를 활용해 수집된 wine database에 필터를 적용하여 특정 와인만 뽑아올 수 있습니다.    
    - input: {body:3, price:100000~120000}
    - output: [Document(page_content='포지오 데이 젤시', metadata={'price': '30000', ...}),]
- 자연어 query로 데이터베이스 정보 추출(추천을 위함): DocArrayInMemorySearch을 활용해 wine database에 자연어로 특정 데이터를 뽑아 올 수 있습니다.
    - input: "프랑스에서 만들어진 와인은?"
- 특정 URL query로 웹 페이지에서 정보 추출(와인 구매링크에서 정보를 가져올 때 사용): SeleniumURLLoader를 이용해 특정 url로 부터 웹페이지 정보를 얻을 수 있습니다.
- 자연어 구글 검색으로 웹 페이지에서 정보 추출(Q&A처럼 그 외 검색이 필요한 경우 사용): SerpAPI를 이용해 구글에서 검색 결과를 가져올 수 있습니다.

#### 당신의 목표는 아래와 같습니다.
1. 다양한 도구를 사용하거나 알고리즘을 탐구하여 데이터베이스를 연결할 수 있는 좋은 방식을 찾습니다. (예를 들면 와인바의 상세정보를 검색하기 위한 카카오 위치 정보 API등을 추가할 수 있을 듯 합니다.)
2. 도구를 사용할 때는 적절한 format을 사용하는 것이 성능에 영향을 줍니다. 각 도구들에 대한 적절한 input format을 찾습니다.


#### 참고사항
현재 말이 너무 모호하게 써있고, 도구의 사용처가 확실하지 않습니다. 이를 구체화하는 과정이 필요하기도 합니다.

-------------------------------------------------------------------------------

### API 키 불러오기

In [1]:
import os
import configparser

In [2]:
config = configparser.ConfigParser()
config.read('./secrets.ini')

['./secrets.ini']

In [3]:
openai_api_key = config['OPENAI']['OPENAI_API_KEY']
serper_api_key = config['SERPER']['SERPER_API_KEY']
serp_api_key = config['SERPAPI']['SERPAPI_API_KEY']
os.environ.update({'OPENAI_API_KEY': openai_api_key})
os.environ.update({'SERPER_API_KEY': serper_api_key})
os.environ.update({'SERPAPI_API_KEY': serp_api_key})

In [4]:
from typing import List, Union
import re
import json

import pandas as pd
from langchain import SerpAPIWrapper, LLMChain
from langchain.agents import Tool, AgentType, AgentExecutor, LLMSingleActionAgent, AgentOutputParser
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain, SimpleSequentialChain
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.document_loaders import DataFrameLoader, SeleniumURLLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.indexes import VectorstoreIndexCreator
from langchain.prompts import PromptTemplate, StringPromptTemplate, load_prompt, BaseChatPromptTemplate
from langchain.llms import OpenAI
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.schema import AgentAction, AgentFinish, HumanMessage
from langchain.vectorstores import DocArrayInMemorySearch, Chroma

### Load wine database json

In [5]:
df = pd.read_json('./data/unified_wine_data.json', encoding='utf-8', lines=True)

In [ ]:
df.head()

### Prepare Langchain Tool

#### Tool1: Wine database 1

In [ ]:
loader =DataFrameLoader(data_frame=df, page_content_column='name')
docs = loader.load()
embeddings = OpenAIEmbeddings()

아래는 wine database1에 metadata_field Attribute이다. 아래를 기준으로 서치를 진행하게 된다.

In [ ]:
metadata_field_info = [
    AttributeInfo(
        name="body",
        description="1-5 rating for the body of wine",
        type="int",
    ),
    AttributeInfo(
        name="sweetness",
        description="1-5 rating for the sweetness of wine",
        type="int",
    ),
    AttributeInfo(
        name="alcohol",
        description="1-5 rating for the alcohol of wine",
        type="int",
    ),
    AttributeInfo(
        name="price",
        description="The price of the wine",
        type="int",
    ),
    AttributeInfo(
        name="rating", 
        description="1-5 rating for the wine", 
        type="float"
    ),
    AttributeInfo(
        name="wine_type", 
        description="The type of wine. It can be '레드', '로제', '스파클링', '화이트', '디저트', '주정강화'", 
        type="string"
    ),
    AttributeInfo(
        name="country", 
        description="The country of wine. It can be '기타 신대륙', '기타구대륙', '뉴질랜드', '독일', '미국', '스페인', '아르헨티나', '이탈리아', '칠레', '포루투칼', '프랑스', '호주'", 
        type="float"
    ),
]

In [ ]:
vectorstore = Chroma.from_documents(docs, embeddings)
document_content_description = "Database of a wine"
llm = OpenAI(temperature=0)
retriever = SelfQueryRetriever.from_llm(
    llm, vectorstore, document_content_description, metadata_field_info, verbose=True
)  # Added missing closing parenthesis


In [ ]:
retriever.get_relevant_documents('{"wine_type":"레드", "body": "lt 3 gt 0"}') # gt means greater than, lt means less than, eq means equal to

query=' ' filter=Operation(operator=<Operator.AND: 'and'>, arguments=[Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='wine_type', value='레드'), Operation(operator=<Operator.AND: 'and'>, arguments=[Comparison(comparator=<Comparator.GT: 'gt'>, attribute='body', value=0), Comparison(comparator=<Comparator.LT: 'lt'>, attribute='body', value=3)])]) limit=None


[Document(page_content='바 독 피노누아', metadata={'url': 'https://www.winenara.com/shop/product/product_view?product_cd=03P976', 'site_name': 'winenara', 'price': 29000, 'en_name': 'BAR DOG PINOT NOIR', 'img_url': 'https://www.winenara.com/uploads/product/550/1936_detail_084.png', 'body': 2, 'acidity': '', 'tannin': '', 'sweetness': -1, 'alcohol': -1, 'wine_type': '레드', 'country': '미국', 'grape': '', 'rating': 3.6, 'pickup_location': '', 'vivino_link': 'https://www.vivino.com/US-CA/en/bar-dog-pinot-noir/w/7129723'}),
 Document(page_content='루이라뚜르 피노누아', metadata={'url': 'https://www.winenara.com/shop/product/product_view?product_cd=03H965', 'site_name': 'winenara', 'price': 52000, 'en_name': 'LOUIS LATOUR PINOT NOIR', 'img_url': 'https://www.winenara.com/uploads/product/550/493_detail_025.png', 'body': 2, 'acidity': '', 'tannin': '', 'sweetness': -1, 'alcohol': -1, 'wine_type': '레드', 'country': '프랑스', 'grape': '', 'rating': 3.6, 'pickup_location': '', 'vivino_link': 'https://www.vivino.com/G

#### Tool2: Search specific wine with url

In [ ]:
def search_with_url(query):
    return SeleniumURLLoader(urls=[query]).load()

#### Tool3: Wine database 2

In [ ]:
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch
).from_loaders([loader])

#### Tool4: Search in Google

In [ ]:
search = SerpAPIWrapper()

In [ ]:
tools = [
    Tool(
        name="Wine database",
        func=retriever.get_relevant_documents,
        description="""
Database about the wines in wine store. You can get information such as the price of the wine, purchase URL, features, rating information, and more.
You can search wines with the following attributes:
- body: 1-5 rating int for the body of wine. You have to specify greater than or less than. For example, if you want to search for wines with a body rating of less than 3, enter 'body: gt 0 lt 3'
- price: The price range of the wine. Please enter the price range in the form of range. For example, if you want to search for wines that cost less than 20,000 won, enter 'price: gt 0 lt20000'
- rating: 1-5 rating float for the wine. You have to specify greater than or less than. For example, if you want to search for wines with a rating of less than 3, enter 'rating: gt 0 lt 3'
- wine_type: The type of wine. It can be '레드', '로제', '스파클링', '화이트', '디저트', '주정강화'
- name: The name of wine. 입력할 때는 '와인 이름은 "비냐 조잘" 입니다' 이런 식으로 입력해주세요.
"""
    ),
    Tool(
        name = "Search specific wine with url",
        func=search_with_url,
        description="Search specific wine with url. Query must be url"
    ),
    Tool(
        name = "Wine database 2",
        func=index.query,
        description="Database about the wines in wine store. You can use this tool if you're having trouble getting information from the wine database tool above. Query must be in String"
    ),
    Tool(
        name = "Search",
        func=search.run,
        description="Useful for when you need to ask with search. Search in English only."
    ),
]